# data import

In [2]:
import pandas as pd

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
wefarm = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/wefarm.parquet")

# peek at the data

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)

wefarm.head(5)

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,response_topic,response_sent,question_user_type,question_user_status,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,None,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',None,2019-01-24 17:54:06.216221+00,farmer,live,ug,None,None,2017-11-18 13:09:11+00,farmer,live,ug,None,None,2017-05-09 09:19:33+00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,None,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,None,2018-01-04 08:57:28+00,farmer,live,ug,None,None,2017-11-20 11:55:48+00,farmer,zombie,ug,None,None,2017-11-22 10:13:03+00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,None,None,2017-08-22 14:51:07+00,farmer,zombie,ug,None,None,2017-08-12 09:30:33+00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,cattle,2017-11-22 12:35:26+00,farmer,zombie,ug,None,None,2017-08-22 14:51:07+00,farmer,zombie,ug,None,None,2017-08-12 09:30:33+00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,None,None,2017-08-22 14:51:07+00,farmer,zombie,ug,None,None,2017-08-12 09:30:33+00


In [6]:
wefarm.shape

(20304843, 24)

# create dataframes by language

In [7]:
wefarm_eng_df = wefarm[wefarm["question_language"] == "eng"][["question_id", "question_content"]].drop_duplicates()

In [8]:
doc_lst = wefarm_eng_df["question_content"]

# install preprocessing libraries libraries

In [9]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 101.2 MB/s eta 0:00:00


In [10]:
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess
from tqdm import tqdm

#preprocessing

In [12]:
lem = WordNetLemmatizer()

def preprocess_text(text):
  tokens = simple_preprocess(text, deacc=True)
  return [lem.lemmatize(word, pos='v') for word in tokens]


In [13]:
processed_questions_eng_lst = []
for question in tqdm(doc_lst):
  processed_questions_eng_lst.append(preprocess_text(question))

100%|██████████| 2940542/2940542 [01:36<00:00, 30591.76it/s]


# keyword extraction

## definition

In [14]:
# keywords manually extracted from previous LDA analysis

financial_inclusion_keywords_lst = [
  "economy",
  "price",
  "sell",
  "market",
  "buy",
  "cost",
  "fee",
  "finance",
  "investment",
  "money",
  "sum",
  "credit",
  "loan",
  "insurance",
  "bank"
]


crop_keywords_lst = [
  "crop",
  "soil",
  "fruit",
  "plant",
  "leave",
  "fertilizer",
  "seed",
  "potatoes",
  "passion",
  "maize",
  "tomatoes",
  "banana",
  "manure",
  "layer",
  "coffee",
  "tea",
  "onions",
  "grow",
  "mulch",
  "bean",
  "type",
  "cabbage",
  "land",
  "variety",
  "yield",
  "keep",
  "rice",
  "harvest",
]

diversification_keywords_lst = [
  "new",
  "expand",
  "begin",
  "best",
  "increase",
  "clear",
  "add",
  "diversify",
  "irrigation",
  "obtain",
  "shift",
  "acquire",
  "scale",
  "register",
  "use",
  "acreage",
  "grow"
]

question_keywords_lst = [
  "who",
  "what",
  "where",
  "when",
  "how",
  "which",
  "many"
]

climate_keywords_lst = [
  "season",
  "time",
  "climate",
  "water",
  "rain",
  "heat",
  "sun",
  "temperature",
  "flood",
  "drought"
]

disease_keywords_lst = [
  "disease",
  "sick",
  "unhealthy",
  "medicine",
  "treat",
  "weed",
  "control",
  "plant",
  "leave",
  "spray",
  "harvest",
  "affect",
  "prevent",
  "pests",
  "chemical",
  "rabbit",
  "care",
  "tick",
  "attack",
  "mean",
  "black",
  "space",
  "turn",
  "keep"
]

livestock_keywords_lst = [
  "livestock",
  "animals",
  "sheep",
  "cattle",
  "donkey",
  "pig",
  "cow",
  "goat",
  "camel",
  "hen",
  "chicken",
  "poultry",
  "lay",
  "egg",
  "milk",
  "breed",
  "dairy",
  "bee"
]

## extrapolation

In [15]:
%%time

from gensim.models import Word2Vec
w2v_model = Word2Vec(processed_questions_eng_lst, min_count=1, vector_size=100)

CPU times: user 3min 5s, sys: 585 ms, total: 3min 5s
Wall time: 1min 8s


In [16]:
def run_w2v_broad(keyword_lst, top_n=50):
  full_keyword_lst = []
  for word in keyword_lst:
    full_keyword_lst.extend([w for w, s in w2v_model.wv.most_similar(word, topn=top_n)])
  return full_keyword_lst

In [17]:
def run_w2v_niche(keyword_lst, top_n=50):
  niche_keyword_dct = {}
  for word in keyword_lst:
    niche_keyword_dct[word] =[w for w, s in w2v_model.wv.most_similar(word, topn=top_n)]
  return niche_keyword_dct

In [18]:
%%time

financial_inclusion_keywords_full_lst = run_w2v_broad(financial_inclusion_keywords_lst)
crop_keywords_full_lst = run_w2v_broad(crop_keywords_lst)
climate_keywords_full_lst = run_w2v_broad(climate_keywords_lst)
disease_keywords_full_lst = run_w2v_broad(disease_keywords_lst)
livestock_keywords_full_lst = run_w2v_broad(livestock_keywords_lst)
diversification_keywords_full_lst = run_w2v_broad(diversification_keywords_lst)
question_keywords_full_lst = run_w2v_broad(question_keywords_lst)

CPU times: user 22.6 s, sys: 10.3 ms, total: 22.6 s
Wall time: 537 ms


In [19]:
broad_category_dct = {
    "financial-inclusion": financial_inclusion_keywords_full_lst,
    "crop": crop_keywords_full_lst,
    "climate": climate_keywords_full_lst,
    "disease": disease_keywords_full_lst,
    "livestock": livestock_keywords_full_lst,
    "diversification": diversification_keywords_full_lst,
    "question": question_keywords_full_lst
}

In [20]:
%%time

financial_inclusion_keywords_niche_dct = run_w2v_niche(financial_inclusion_keywords_lst)
crop_keywords_niche_dct = run_w2v_niche(crop_keywords_lst)
climate_keywords_niche_dct = run_w2v_niche(climate_keywords_lst)
disease_keywords_niche_dct = run_w2v_niche(disease_keywords_lst)
livestock_keywords_niche_dct = run_w2v_niche(livestock_keywords_lst)
diversification_keywords_niche_dct = run_w2v_niche(diversification_keywords_lst)
question_keywords_niche_dct = run_w2v_niche(question_keywords_lst)

CPU times: user 23.6 s, sys: 469 µs, total: 23.6 s
Wall time: 538 ms


In [21]:
niche_category_dct = {
  "financial-inclusion": financial_inclusion_keywords_niche_dct,
  "crop": crop_keywords_niche_dct,
  "climate": climate_keywords_niche_dct,
  "disease": disease_keywords_niche_dct,
  "livestock": livestock_keywords_niche_dct,
  "diversification": diversification_keywords_niche_dct,
  "question": question_keywords_niche_dct
}

# text categorization

## niche keywords

In [22]:
def find_niche_keywords(doc):
  doc_words = set(doc)
  matches_dct = {}
  for category, sub_dct in niche_category_dct.items():
    for sub_category, keywords in sub_dct.items():
      if any(keyword in doc_words for keyword in keywords):
        matches_dct[f"{category}_{sub_category}"] = 1
  return matches_dct

In [23]:
niche_res_lst = []

for doc in tqdm(processed_questions_eng_lst):
  matches_dct = find_niche_keywords(doc)
  niche_res_lst.append(matches_dct)

100%|██████████| 2940542/2940542 [11:27<00:00, 4277.43it/s]


In [24]:
%%time

niche_res_df = pd.DataFrame(niche_res_lst)

CPU times: user 21.5 s, sys: 1.33 s, total: 22.8 s
Wall time: 22.8 s


In [25]:
joined_niche_questions_eng_lst = [" ".join(doc) for doc in processed_questions_eng_lst]
niche_questions_eng_df = pd.DataFrame({
  "preprocessed_question": joined_niche_questions_eng_lst
})

In [26]:
niche_questions_id_eng_df = pd.concat([niche_questions_eng_df.reset_index(drop=True), wefarm_eng_df.reset_index(drop=True)], axis=1)
niche_questions_categories_df = pd.concat([niche_questions_id_eng_df, niche_res_df], axis=1)

In [27]:
niche_questions_categories_df.head(5)

,preprocessed_question,question_id,question_content,financial-inclusion_price,financial-inclusion_sell,financial-inclusion_buy,diversification_obtain,diversification_acquire,question_what,question_when,question_how,question_which,crop_harvest,climate_time,disease_harvest,livestock_livestock,livestock_animals,livestock_sheep,livestock_cattle,livestock_donkey,livestock_pig,livestock_cow,livestock_goat,livestock_camel,livestock_hen,livestock_chicken,livestock_poultry,livestock_bee,diversification_expand,diversification_begin,crop_plant,crop_variety,disease_plant,crop_seed,crop_grow,crop_mulch,diversification_grow,crop_crop,crop_potatoes,crop_passion,crop_maize,crop_banana,crop_coffee,crop_tea,crop_onions,crop_bean,crop_cabbage,climate_rain,diversification_add,question_who,disease_rabbit,diversification_use,crop_tomatoes,crop_rice,disease_disease,disease_care,crop_keep,disease_keep,financial-inclusion_market,financial-inclusion_cost,question_many,crop_layer,disease_treat,disease_prevent,disease_chemical,crop_fruit,livestock_milk,crop_leave,crop_land,disease_leave,livestock_egg,livestock_breed,crop_manure,diversification_increase,climate_season,disease_control,climate_sun,crop_yield,livestock_dairy,crop_soil,diversification_irrigation,diversification_scale,financial-inclusion_money,question_where,diversification_best,disease_spray,disease_turn,disease_attack,financial-inclusion_fee,crop_type,climate_climate,disease_medicine,disease_tick,disease_weed,disease_pests,financial-inclusion_finance,financial-inclusion_investment,financial-inclusion_loan,diversification_new,livestock_lay,climate_heat,climate_drought,diversification_clear,disease_black,disease_space,crop_fertilizer,disease_sick,climate_temperature,climate_flood,disease_affect,financial-inclusion_sum,disease_unhealthy,financial-inclusion_bank,financial-inclusion_insurance,disease_mean,financial-inclusion_credit,diversification_acreage,diversification_diversify,diversification_register,climate_water,diversification_shift,financial-inclusion_economy
0,this go to wefarm be it possible to get for us...,3849061,Q this goes to wefarm. is it possible to get f...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,have stock rabbit urine for weeks mashambani s...,3849084,Q-i have stock rabbit's urine for 5 weeks mash...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,have mi can start aproject of poutry how can d...,3849098,Q J Have Mi 10000 Can J Start Aproject Of Pout...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,where do get seed of coconut,3849100,WHERE DO I GET SEEDS OF COCONUT?,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,

In [28]:
keyword_collst = niche_questions_categories_df.columns[3:]
keyword_sums = niche_questions_categories_df[keyword_collst].sum().sort_values()
sums_df = keyword_sums.reset_index()
sums_df.columns = ["col", "sum"]

sums_df["prefix"] = [c.split("_")[0] for c in sums_df["col"]]

sums_df = sums_df.sort_values(["prefix", "sum"], ascending=[True, False])

keyword_sums_dct = dict(zip(sums_df["col"], sums_df["sum"]))

In [29]:
import plotly.graph_objects as go

fig = go.Figure()

for k, v in keyword_sums_dct.items():
  fig.add_trace(
    go.Bar(
      x=[k],
      y=[v],
      marker={
        "color": [v],
        "colorscale": "greens"
      },
      name=k,
    )
  )

fig

In [30]:
niche_questions_categories_df.to_parquet("en_questions_cat_niche.parquet", index=False)
!cp en_questions_cat_niche.parquet /content/drive/MyDrive/DataKit_WeFarm

## broad keywords

In [31]:
def find_keywords(doc):
  doc_words = set(doc)
  matches_dct = {}
  for category, keywords in broad_category_dct.items():
    if any(keyword in doc_words for keyword in keywords):
      matches_dct[category] = 1
  return matches_dct

In [32]:
broad_res_lst = []

for doc in tqdm(processed_questions_eng_lst):
  matches_dct = find_keywords(doc)
  broad_res_lst.append(matches_dct)

100%|██████████| 2940542/2940542 [06:33<00:00, 7470.44it/s]


In [33]:
broad_res_df = pd.DataFrame(broad_res_lst)

In [34]:
broad_joined_questions_eng_lst = [" ".join(doc) for doc in processed_questions_eng_lst]
broad_questions_eng_df = pd.DataFrame({
  "preprocessed_question": broad_joined_questions_eng_lst
})

In [35]:
broad_questions_id_eng_df = pd.concat([broad_questions_eng_df.reset_index(drop=True), wefarm_eng_df.reset_index(drop=True)], axis=1)
broad_questions_categories_df = pd.concat([broad_questions_id_eng_df, broad_res_df], axis=1)

In [36]:
broad_questions_categories_df.head(5)

,preprocessed_question,question_id,question_content,financial-inclusion,diversification,question,crop,climate,disease,livestock
0,this go to wefarm be it possible to get for us...,3849061,Q this goes to wefarm. is it possible to get f...,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,have stock rabbit urine for weeks mashambani s...,3849084,Q-i have stock rabbit's urine for 5 weeks mash...,NaN,NaN,1.0,1.0,1.0,1.0,1.0
2,have mi can start aproject of poutry how can d...,3849098,Q J Have Mi 10000 Can J Start Aproject Of Pout...,NaN,1.0,1.0,NaN,NaN,NaN,1.0
3,where do get seed of coconut,3849100,WHERE DO I GET SEEDS OF COCONUT?,1.0,1.0,1.0,1.0,NaN,1.0,NaN
4,which plant have omega,3849129,Q#.Which plant has omega3?,NaN,1.0,1.0,1.0,NaN,1.0,NaN


In [38]:
sum_financial_inclusion = broad_questions_categories_df["financial-inclusion"].sum()
sum_livestock = broad_questions_categories_df["livestock"].sum()
sum_crop = broad_questions_categories_df["crop"].sum()
sum_disease = broad_questions_categories_df["disease"].sum()
sum_climate = broad_questions_categories_df["climate"].sum()

In [39]:
count_dct = {
  "financial_inclusion": sum_financial_inclusion,
  "livestock": sum_livestock,
  "crop": sum_crop,
  "disease": sum_disease,
  "climate": sum_climate,
}

In [40]:
broad_questions_categories_df.to_parquet("en_questions_cat_broad.parquet", index=False)
!cp en_questions_cat_broad.parquet /content/drive/MyDrive/DataKit_WeFarm

In [ ]:
# broad_questions_categories_df[["market", "disease", "crop", "seasonality", "livestock"]].isnull().all(axis=1).sum()